<a href="https://colab.research.google.com/github/matdcp/Data-dreamers-/blob/main/TRANSFORMER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upload cleaned dataset

In [2]:
import pandas as pd
from google.colab import files

# Carica il file CSV
uploaded = files.upload()

Saving cleaned_dataset_NN.csv to cleaned_dataset_NN.csv


In [3]:
# Leggi il file CSV con Pandas
for filename in uploaded.keys():
    df = pd.read_csv(filename)

df.head()

,id,Start,End,Label
0,Ventricular,0,11,B-Multi-tissue_structure
1,fibrillation,12,24,O
2,long,32,36,O
3,QT,37,39,O
4,syndrome,40,48,O


In [4]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00


In [6]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ea8c95f5a3319e56c934440350683cc7693d0d679588678e69f4eb0d2f1d05db
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from transformers import AutoTokenizer
from datasets import Dataset
import os


#Splitting the dataset and rebalancing classes

In [8]:
# Valori unici di Label
print("\nCategorie di Label:")
print(df['Label'].unique())

# Frequenza delle etichette
print("\nDistribuzione delle etichette:")
print(df['Label'].value_counts())



Categorie di Label:
['B-Multi-tissue_structure' 'O' 'B-Organism_substance'
 'B-Organism_subdivision' 'B-Organ' 'I-Multi-tissue_structure'
 'B-Cellular_component' 'I-Cellular_component' 'B-Cell' 'I-Cell'
 'B-Immaterial_anatomical_entity' 'B-Tissue' 'I-Tissue'
 'B-Pathological_formation' 'B-Anatomical_system' 'I-Organism_substance'
 'I-Anatomical_system' 'I-Pathological_formation'
 'I-Immaterial_anatomical_entity' 'I-Organ' 'I-Organism_subdivision'
 'B-Developing_anatomical_structure' 'I-Developing_anatomical_structure']

Distribuzione delle etichette:
Label
O                                    56455
B-Cell                                 775
B-Multi-tissue_structure               639
I-Cell                                 439
B-Organ                                381
B-Pathological_formation               367
I-Multi-tissue_structure               358
B-Organism_substance                   288
I-Pathological_formation               224
B-Cellular_component                   199
B-Tiss

In [9]:
id_label_counts = df.groupby('id')['Label'].nunique()
print(id_label_counts[id_label_counts > 1])

id
1         2
12        2
16        2
17        2
2         3
         ..
walls     2
white     5
window    2
yolk      3
zones     2
Name: Label, Length: 491, dtype: int64


In [10]:
labels = df.groupby('id')['Label'].first()
print(labels.value_counts())

Label
O                                    11938
B-Multi-tissue_structure               159
B-Cell                                 108
B-Pathological_formation                93
I-Multi-tissue_structure                82
B-Organ                                 60
B-Organism_substance                    47
I-Cell                                  42
B-Tissue                                42
B-Organism_subdivision                  41
B-Cellular_component                    40
I-Pathological_formation                40
I-Tissue                                19
I-Organism_substance                    14
B-Immaterial_anatomical_entity          14
I-Cellular_component                    14
I-Organ                                 13
B-Anatomical_system                     11
B-Developing_anatomical_structure        9
I-Immaterial_anatomical_entity           4
I-Organism_subdivision                   3
I-Anatomical_system                      2
I-Developing_anatomical_structure        1
Name:

In [11]:
from sklearn.model_selection import train_test_split

# Rimuovere la classe con 1 occorrenza (I-Developing_anatomical_structure)
df = df[~df['Label'].isin(['I-Developing_anatomical_structure'])]
# corrispondenza univoca id → Label
labels = df.groupby('id')['Label'].first()

# Suddivisione del dataset con stratificazione basata sulla colonna "label"
unique_ids = labels.index

train_ids, temp_ids = train_test_split(unique_ids,test_size=0.3,random_state=42,stratify=labels)

temp_labels = labels[temp_ids] #label associated to temporary ID

# Filtro per garantire almeno 2 occorrenze per classe in temp_labels
valid_temp_labels = temp_labels.value_counts()[temp_labels.value_counts() >= 2].index
temp_ids = temp_ids[temp_labels.isin(valid_temp_labels)]  # Filter valid IDs
temp_labels = temp_labels[temp_labels.isin(valid_temp_labels)]  # update valid lables

# Suddivisione di temp in validation (15%) e test (15%) con stratificazione
val_ids, test_ids = train_test_split(temp_ids,test_size=0.5,random_state=42,stratify=temp_labels)

# Creazione dei DataFrame per ogni set
train_df = df[df['id'].isin(train_ids)]
val_df = df[df['id'].isin(val_ids)]
test_df = df[df['id'].isin(test_ids)]

print("Dimensioni dei set:")
print("Train set:", train_df.shape)
print("Validation set:", val_df.shape)
print("Test set:", test_df.shape)


Dimensioni dei set:
Train set: (42471, 4)
Validation set: (9538, 4)
Test set: (8956, 4)


##rebalncing training set

In [12]:
# Bilancia solo il training set
balanced_train_df = pd.DataFrame()
for label in train_df['Label'].unique():
    subset = train_df[train_df['Label'] == label]
    if len(subset) < train_df['Label'].value_counts().max():
        subset = resample(subset, replace=True, n_samples=train_df['Label'].value_counts().max(), random_state=42)
    balanced_train_df = pd.concat([balanced_train_df, subset])

# Shuffle per mescolare i dati bilanciati
balanced_train_df = balanced_train_df.sample(frac=1, random_state=42).reset_index(drop=True)

print("\nDistribuzione delle etichette nel training set bilanciato:")
print(balanced_train_df['Label'].value_counts())


Distribuzione delle etichette nel training set bilanciato:
Label
I-Tissue                             39268
B-Pathological_formation             39268
B-Tissue                             39268
I-Organism_subdivision               39268
B-Organ                              39268
I-Cellular_component                 39268
B-Cellular_component                 39268
B-Organism_substance                 39268
I-Organism_substance                 39268
B-Developing_anatomical_structure    39268
I-Immaterial_anatomical_entity       39268
I-Multi-tissue_structure             39268
O                                    39268
B-Organism_subdivision               39268
B-Immaterial_anatomical_entity       39268
I-Cell                               39268
I-Organ                              39268
I-Anatomical_system                  39268
B-Cell                               39268
B-Anatomical_system                  39268
B-Multi-tissue_structure             39268
I-Pathological_formation       

#Mapping NER tags

In [13]:
# Creiamo una lista di tutte le etichette presenti nel training set bilanciato
all_labels = balanced_train_df['Label'].unique()

# Creiamo il mapping delle etichette
tag2id = {tag: idx for idx, tag in enumerate(all_labels)}
id2tag = {idx: tag for tag, idx in tag2id.items()}

# Aggiungiamo la colonna con l'ID delle etichette nei dataset
balanced_train_df['Label_ID'] = balanced_train_df['Label'].map(tag2id)
val_df['Label_ID'] = val_df['Label'].map(tag2id)
test_df['Label_ID'] = test_df['Label'].map(tag2id)

# Controlliamo il mapping
print("Mapping delle etichette (tag2id):", tag2id)


Mapping delle etichette (tag2id): {'I-Tissue': 0, 'B-Pathological_formation': 1, 'B-Tissue': 2, 'I-Organism_subdivision': 3, 'B-Organ': 4, 'I-Cellular_component': 5, 'B-Cellular_component': 6, 'B-Organism_substance': 7, 'I-Organism_substance': 8, 'B-Developing_anatomical_structure': 9, 'I-Immaterial_anatomical_entity': 10, 'I-Multi-tissue_structure': 11, 'O': 12, 'B-Organism_subdivision': 13, 'B-Immaterial_anatomical_entity': 14, 'I-Cell': 15, 'I-Organ': 16, 'I-Anatomical_system': 17, 'B-Cell': 18, 'B-Anatomical_system': 19, 'B-Multi-tissue_structure': 20, 'I-Pathological_formation': 21}


<ipython-input-13-a2a10882e578>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df['Label_ID'] = val_df['Label'].map(tag2id)
<ipython-input-13-a2a10882e578>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['Label_ID'] = test_df['Label'].map(tag2id)


#Preparing Data for Hugging Face

In [14]:
# Trasforma le etichette in ID numerici usando il mapping tag2id
def convert_labels_to_ids(df, tag2id):
    df.loc[:,'Label_ID'] = df['Label'].map(tag2id)  # Mappa direttamente i valori della colonna
    return df

# Esegui la conversione per ogni dataset
train_dataset = convert_labels_to_ids(train_df, tag2id)
val_dataset = convert_labels_to_ids(val_df, tag2id)
test_dataset = convert_labels_to_ids(test_df, tag2id)

print("Conversione delle etichette completata!")
print(train_dataset.head())

Conversione delle etichette completata!
            id  Start  End                     Label  Label_ID
0  Ventricular      0   11  B-Multi-tissue_structure        20
2         long     32   36                         O        12
3           QT     37   39                         O        12
4     syndrome     40   48                         O        12
5     probably     49   57                         O        12


<ipython-input-14-bcd5c7678cc0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'Label_ID'] = df['Label'].map(tag2id)  # Mappa direttamente i valori della colonna


#Dataset for Hugging Face

In [15]:
# Converto i DataFrame in Dataset di Hugging Face
train_hf = Dataset.from_pandas(train_dataset)
val_hf = Dataset.from_pandas(val_dataset)
test_hf = Dataset.from_pandas(test_dataset)

#Loading tokenizer and pre-trained model

In [16]:
# Carichiamo il tokenizer di DistilBERT
model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Tokenizer caricato:", model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Tokenizer caricato: distilbert-base-cased


#Tokenization and labels allignment

In [17]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['id'],                # Testo da tokenizzare
        truncation=True,               # Tronca se necessario
        padding='max_length',          # Applica padding
        max_length=128,                # Lunghezza massima
        return_offsets_mapping=True,   # Mappa gli offset per i token
        is_split_into_words=False      # I token non sono pre-suddivisi
    )

    labels = []
    for i, label in enumerate(examples['Label_ID']):
        word_ids = tokenized_inputs.offset_mapping[i]
        aligned_labels = []
        for idx, word_id in enumerate(word_ids):
            if word_id is None or tokenized_inputs.input_ids[i][idx] == tokenizer.pad_token_id:
                aligned_labels.append(-100)  # Ignora token speciali e padding
            else:
                aligned_labels.append(label)  # Usa l'etichetta originale
        labels.append(aligned_labels)

    tokenized_inputs.pop("offset_mapping")  # Rimuove offset_mapping
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [18]:
# Applico la tokenizzazione ai dataset
def process_datasets(dataset, tokenizer):
    return dataset.map(
        tokenize_and_align_labels,
        batched=True,  # Processa in batch per velocità e robustezza
        remove_columns=['id', 'Label', 'Label_ID']  # Rimuove colonne inutili
    )


# Tokenizzazione per i tre dataset
tokenized_datasets = {}
for split, dataset in zip(["train", "validation", "test"], [train_hf, val_hf, test_hf]):
    tokenized_datasets[split] = process_datasets(dataset, tokenizer)

print("Tokenizzazione completata!")



Map:   0%|          | 0/42471 [00:00<?, ? examples/s]

Map:   0%|          | 0/9538 [00:00<?, ? examples/s]

Map:   0%|          | 0/8956 [00:00<?, ? examples/s]

Tokenizzazione completata!


In [19]:
# Controlla un esempio del dataset tokenizzato
print("Esempio tokenizzato (Train):")
print(tokenized_datasets["train"].select(range(5)).to_dict())


Esempio tokenizzato (Train):
{'Start': [0, 32, 37, 40, 49], 'End': [11, 36, 39, 48, 57], '__index_level_0__': [0, 2, 3, 4, 5], 'input_ids': [[101, 159, 3452, 4907, 5552, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 1263, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 154, 1942, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

#Model Configuration

In [20]:
from transformers import AutoModelForTokenClassification

# Carichiamo il modello DistilBERT configurato per il token classification
model = AutoModelForTokenClassification.from_pretrained(
    model_name,           # Nome del modello pre-addestrato
    num_labels=len(tag2id)  # Numero di classi (etichette uniche)
)

print("Modello caricato con successo!")


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modello caricato con successo!


#Preparing for training
##Data collator

In [21]:
from transformers import DataCollatorForTokenClassification

# Configuriamo il Data Collator per il task di classificazione dei token
data_collator = DataCollatorForTokenClassification(tokenizer)
print("Data Collator configurato!")


Data Collator configurato!


In [22]:
import evaluate
import numpy as np

# Carichiamo la libreria per le metriche
seqeval = evaluate.load("seqeval")

# Funzione per calcolare le metriche
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)  # Convertiamo le predizioni in classi

    # Escludiamo token speciali (-100)
    true_predictions = [
        [id2tag[p] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]
    true_labels = [
        [id2tag[l] for (p, l) in zip(pred, label) if l != -100]
        for pred, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

print("Metriche configurate!")


Metriche configurate!


##Setting training parameters

In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",          # Directory di output per i risultati
    num_train_epochs=5,              # Numero di epoche
    per_device_train_batch_size=64,  # Batch size per l'addestramento
    per_device_eval_batch_size=128,   # Batch size per la validazione
    eval_strategy="epoch",     # Valutazione alla fine di ogni epoca
    save_strategy="epoch",           # Salvataggio alla fine di ogni epoca
    load_best_model_at_end=True,     # Carica il modello migliore alla fine
    metric_for_best_model="f1",      # Utilizza F1 come metrica principale
    report_to="none"                 # Disabilita Weights & Biases
)

print("Argomenti di addestramento configurati!")


Argomenti di addestramento configurati!


#Trainer

In [24]:
from transformers import Trainer, EarlyStoppingCallback

# Configura il Trainer
trainer = Trainer(
    model=model,                              # Modello configurato
    args=training_args,                       # Argomenti di addestramento
    train_dataset=tokenized_datasets["train"],  # Dataset di addestramento
    eval_dataset=tokenized_datasets["validation"],  # Dataset di validazione
    data_collator=data_collator,              # Data Collator
    compute_metrics=compute_metrics,          # Metriche
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]  # Early stopping
)

print("Trainer configurato con successo!")



Trainer configurato con successo!


#TRAINING

In [25]:
print(tokenized_datasets["train"][0])


{'Start': 0, 'End': 11, '__index_level_0__': 0, 'input_ids': [101, 159, 3452, 4907, 5552, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': [20, 20, 20, 20, 20, 20, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -10

In [26]:
# Preleva un batch di esempio
sample_batch = tokenized_datasets["train"][:2]

# Converte il batch in una lista di dizionari
sample_batch_list = [{key: sample_batch[key][i] for key in sample_batch} for i in range(len(sample_batch["input_ids"]))]

# Applica il data collator
processed_batch = data_collator(sample_batch_list)

print("Batch processato dal data collator:")
print(processed_batch)


Batch processato dal data collator:
{'Start': tensor([ 0, 32]), 'End': tensor([11, 36]), '__index_level_0__': tensor([0, 2]), 'input_ids': tensor([[ 101,  159, 3452, 4907, 5552,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0, 

In [27]:
# Avvia l'addestramento del modello
print("Inizio dell'addestramento...")
trainer.train()
print("Addestramento completato!")


Inizio dell'addestramento...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.309300,0.365665,0.338308,0.140845,0.198889,0.926013
2,0.182700,0.332158,0.433523,0.316073,0.365597,0.930924
3,0.141600,0.350458,0.492424,0.296189,0.369891,0.933206
4,0.101400,0.381017,0.515302,0.299917,0.379157,0.932831
5,0.086300,0.374145,0.474367,0.318144,0.380858,0.933668


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Addestramento completato!


In [30]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# Directory di salvataggio
save_directory = '/content/drive/MyDrive/saved_model'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Salva il modello e il tokenizer
trainer.save_model(save_directory)
tokenizer.save_pretrained(save_directory)

print(f"Modello e tokenizer salvati nella directory: {save_directory}")


Mounted at /content/drive
Modello e tokenizer salvati nella directory: /content/drive/MyDrive/saved_model
